In [1]:
# Import libraries
# import warnings
# warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns

pd.set_option('display.max_columns', 500)

###Data Import

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
# Import train and test data
beneficiary_test = pd.read_csv("/content/gdrive/MyDrive/BT4012/Data/Test_Beneficiarydata-1542969243754.csv")
inpatient_test = pd.read_csv("/content/gdrive/MyDrive/BT4012/Data/Test_Inpatientdata-1542969243754.csv")
outpatient_test = pd.read_csv("/content/gdrive/MyDrive/BT4012/Data/Test_Outpatientdata-1542969243754.csv")
test = pd.read_csv("/content/gdrive/MyDrive/BT4012/Data/Test-1542969243754.csv")

beneficiary_train = pd.read_csv("/content/gdrive/MyDrive/BT4012/Data/Train_Beneficiarydata-1542865627584.csv")
inpatient_train = pd.read_csv("/content/gdrive/MyDrive/BT4012/Data/Train_Inpatientdata-1542865627584.csv")
outpatient_train = pd.read_csv("/content/gdrive/MyDrive/BT4012/Data/Train_Outpatientdata-1542865627584.csv")
train = pd.read_csv("/content/gdrive/MyDrive/BT4012/Data/Train-1542865627584.csv")

print("Data Read")

Data Read


###Merging and Preprocessing

In [4]:
# create binary columns to indicate Inpatient status (1)
inpatient_train['is_inpatient'] = 1
outpatient_train['is_inpatient'] = 0

inpatient_test['is_inpatient'] = 1
outpatient_test['is_inpatient'] = 0

In [5]:
# concat inpatient and outpatient datasets to form a single patient column
patient_train = pd.concat([inpatient_train, outpatient_train])

patient_test = pd.concat([inpatient_test, outpatient_test])

In [6]:
# left join with beneficiary data on BeneID
patient_train = pd.merge(patient_train, beneficiary_train, how='left', on='BeneID')

patient_test = pd.merge(patient_test, beneficiary_test, how='left', on='BeneID')

In [7]:
# left join with label on Provider
patient_train = pd.merge(patient_train, train, how='left', on='Provider')

patient_test = pd.merge(patient_test, test, how='left', on='Provider')

In [8]:
print(f"Shape of train data: {patient_train.shape}")
print(f"Shape of test data: {patient_test.shape}")
# Test data is not labelled

Shape of train data: (558211, 56)
Shape of test data: (135392, 55)


####Extracting the diagnosis codes and generating embeddings

In [9]:
patient_train.head()

,BeneID,ClaimID,ClaimStartDt,ClaimEndDt,Provider,InscClaimAmtReimbursed,AttendingPhysician,OperatingPhysician,OtherPhysician,AdmissionDt,ClmAdmitDiagnosisCode,DeductibleAmtPaid,DischargeDt,DiagnosisGroupCode,ClmDiagnosisCode_1,ClmDiagnosisCode_2,ClmDiagnosisCode_3,ClmDiagnosisCode_4,ClmDiagnosisCode_5,ClmDiagnosisCode_6,ClmDiagnosisCode_7,ClmDiagnosisCode_8,ClmDiagnosisCode_9,ClmDiagnosisCode_10,ClmProcedureCode_1,ClmProcedureCode_2,ClmProcedureCode_3,ClmProcedureCode_4,ClmProcedureCode_5,ClmProcedureCode_6,is_inpatient,DOB,DOD,Gender,Race,RenalDiseaseIndicator,State,County,NoOfMonths_PartACov,NoOfMonths_PartBCov,ChronicCond_Alzheimer,ChronicCond_Heartfailure,ChronicCond_KidneyDisease,ChronicCond_Cancer,ChronicCond_ObstrPulmonary,ChronicCond_Depression,ChronicCond_Diabetes,ChronicCond_IschemicHeart,ChronicCond_Osteoporasis,ChronicCond_rheumatoidarthritis,ChronicCond_stroke,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt,PotentialFraud
0,BENE11001,CLM46614,2009-04-12,2009-04-18,PRV55912,26000,PHY390922,NaN,NaN,2009-04-12,7866,1068.0,2009-04-18,201,1970,4019,5853,7843,2768,71590,2724,19889,5849,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1943-01-01,NaN,1,1,0,39,230,12,12,1,2,1,2,2,1,1,1,2,1,1,36000,3204,60,70,Yes
1,BENE11001,CLM66048,2009-08-31,2009-09-02,PRV55907,5000,PHY318495,PHY318495,NaN,2009-08-31,6186,1068.0,2009-09-02,750,6186,2948,56400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7092.0,NaN,NaN,NaN,NaN,NaN,1,1943-01-01,NaN,1,1,0,39,230,12,12,1,2,1,2,2,1,1,1,2,1,1,36000,3204,60,70,No
2,BENE11001,CLM68358,2009-09-17,2009-09-20,PRV56046,5000,PHY372395,NaN,PHY324689,2009-09-17,29590,1068.0,2009-09-20,883,29623,30390,71690,34590,V1581,32723,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1943-01-01,NaN,1,1,0,39,230,12,12,1,2,1,2,2,1,1,1,2,1,1,36000,3204,60,70,No
3,BENE11011,CLM38412,2009-02-14,2009-02-22,PRV52405,5000,PHY369659,PHY392961,PHY349768,2009-02-14,431,1068.0,2009-02-22,067,43491,2762,7843,32723,V1041,4254,25062,40390,4019,NaN,331.0,NaN,NaN,NaN,NaN,NaN,1,1914-03-01,NaN,2,2,0,1,360,12,12,2,1,1,2,2,1,1,2,2,1,1,5000,1068,250,320,No
4,BENE11014,CLM63689,2009-08-13,2009-08-30,PRV56614,10000,PHY379376,PHY398258,NaN,2009-08-13,78321,1068.0,2009-08-30,975,042,3051,34400,5856,42732,486,5119,29620,20300,NaN,3893.0,NaN,NaN,NaN,NaN,NaN,1,1938-04-01,NaN,2,1,Y,45,780,12,12,2,1,1,2,1,1,2,1,2,2,2,21260,2136,120,100,No


In [10]:
cols_of_interest = ['Provider', 'ClmDiagnosisCode_1', 'ClmDiagnosisCode_2', 'ClmDiagnosisCode_3', 'ClmDiagnosisCode_4', 'ClmDiagnosisCode_5', 'ClmDiagnosisCode_6', 'ClmDiagnosisCode_7', 'ClmDiagnosisCode_8',
                    'ClmDiagnosisCode_9', 'ClmDiagnosisCode_10']

diag_train = patient_train[cols_of_interest]
diag_test = patient_test[cols_of_interest]

diag_train.head()

,Provider,ClmDiagnosisCode_1,ClmDiagnosisCode_2,ClmDiagnosisCode_3,ClmDiagnosisCode_4,ClmDiagnosisCode_5,ClmDiagnosisCode_6,ClmDiagnosisCode_7,ClmDiagnosisCode_8,ClmDiagnosisCode_9,ClmDiagnosisCode_10
0,PRV55912,1970,4019,5853,7843,2768,71590,2724,19889,5849,NaN
1,PRV55907,6186,2948,56400,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,PRV56046,29623,30390,71690,34590,V1581,32723,NaN,NaN,NaN,NaN
3,PRV52405,43491,2762,7843,32723,V1041,4254,25062,40390,4019,NaN
4,PRV56614,042,3051,34400,5856,42732,486,5119,29620,20300,NaN


In [11]:
# Impute all the NaNs with empty strings
diag_train.fillna('', inplace=True)
diag_test.fillna('', inplace=True)

<ipython-input-11-26d0c3a2d171>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  diag_train.fillna('', inplace=True)
<ipython-input-11-26d0c3a2d171>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  diag_test.fillna('', inplace=True)


In [12]:
# Read in the diagnosis code dataset
cols_to_read = ['DIAGNOSIS CODE', 'SHORT DESCRIPTION']

diag_codes = pd.read_excel('diag_codes.xlsx', usecols=cols_to_read)
diag_codes = diag_codes.set_index('DIAGNOSIS CODE')
diag_dict = diag_codes['SHORT DESCRIPTION'].to_dict()
# diag_dict

In [13]:
# Convert all the diagnosis codes in the diag dataframes into their descriptions
diag_train = diag_train.applymap(lambda x : diag_dict.get(x, x))
diag_test = diag_test.applymap(lambda x : diag_dict.get(x, x))

print("Diagnosis codes mapped")

Diagnosis codes mapped


In [14]:
diag_train.head()

,Provider,ClmDiagnosisCode_1,ClmDiagnosisCode_2,ClmDiagnosisCode_3,ClmDiagnosisCode_4,ClmDiagnosisCode_5,ClmDiagnosisCode_6,ClmDiagnosisCode_7,ClmDiagnosisCode_8,ClmDiagnosisCode_9,ClmDiagnosisCode_10
0,PRV55912,Secondary malig neo lung,Hypertension NOS,Chr kidney dis stage III,Aphasia,Hypopotassemia,Osteoarthros NOS-unspec,Hyperlipidemia NEC/NOS,Secondary malig neo NEC,Acute kidney failure NOS,
1,PRV55907,Vaginal enterocele,Mental disor NEC oth dis,Constipation NOS,,,,,,,
2,PRV56046,Depress psychosis-severe,Alcoh dep NEC/NOS-unspec,Arthropathy NOS-unspec,Epilep NOS w/o intr epil,Hx of past noncompliance,Obstructive sleep apnea,,,,
3,PRV52405,Crbl art ocl NOS w infrc,Acidosis,Aphasia,Obstructive sleep apnea,Hx-cervical malignancy,Prim cardiomyopathy NEC,DMII neuro uncntrld,Hy kid NOS w cr kid I-IV,Hypertension NOS,
4,PRV56614,Human immuno virus dis,Tobacco use disorder,"Quadriplegia, unspecifd",End stage renal disease,Atrial flutter,"Pneumonia, organism NOS",Pleural effusion NOS,Depress psychosis-unspec,Mult mye w/o achv rmson,


In [15]:
clm_codes = ['ClmDiagnosisCode_1', 'ClmDiagnosisCode_2', 'ClmDiagnosisCode_3', 'ClmDiagnosisCode_4', 'ClmDiagnosisCode_5', 'ClmDiagnosisCode_6', 'ClmDiagnosisCode_7', 'ClmDiagnosisCode_8',
              'ClmDiagnosisCode_9', 'ClmDiagnosisCode_10']

# Combine all the column values into a single column via string concatenation
diag_train['Comb_Diagnoses'] = diag_train[clm_codes].apply(lambda x: ', '.join(x), axis=1)
diag_test['Comb_Diagnoses'] = diag_test[clm_codes].apply(lambda x: ', '.join(x), axis=1)

# Drop all the initial diagnosis features
diag_train = diag_train.drop(clm_codes, axis=1)
diag_test = diag_test.drop(clm_codes, axis=1)

diag_train.head()

,Provider,Comb_Diagnoses
0,PRV55912,"Secondary malig neo lung, Hypertension NOS, Ch..."
1,PRV55907,"Vaginal enterocele, Mental disor NEC oth dis, ..."
2,PRV56046,"Depress psychosis-severe, Alcoh dep NEC/NOS-un..."
3,PRV52405,"Crbl art ocl NOS w infrc, Acidosis, Aphasia, O..."
4,PRV56614,"Human immuno virus dis, Tobacco use disorder, ..."


####Split the data into training and validation data for model building

In [16]:
from sklearn.model_selection import GroupShuffleSplit

gss = GroupShuffleSplit(n_splits=2, test_size=.2, random_state=4012)
split = gss.split(diag_train, groups=diag_train.Provider)
train_ix, val_ix = next(split)

X_diag_train = diag_train.loc[train_ix]
X_diag_val = diag_train.loc[val_ix]

providers_train = X_diag_train.Provider.unique()
providers_val = X_diag_val.Provider.unique()

In [17]:
X_diag_val.head()

,Provider,Comb_Diagnoses
0,PRV55912,"Secondary malig neo lung, Hypertension NOS, Ch..."
4,PRV56614,"Human immuno virus dis, Tobacco use disorder, ..."
5,PRV54986,"Mal neo breast low-outer, Acq absnce breast/ni..."
10,PRV55193,"E coli septicemia, Glucocorticoid deficient, S..."
13,PRV52283,"Obs chr bronc w(ac) exac, Dis magnesium metabo..."


####Aggregate over Provider, and merge the diagnoses

In [18]:
# Aggregate the training data
grouped_diag_train = X_diag_train.groupby('Provider')['Comb_Diagnoses'].agg(lambda x: '; '.join(x))
grouped_diag_val = X_diag_val.groupby('Provider')['Comb_Diagnoses'].agg(lambda x: '; '.join(x))

grouped_diag_train.loc['PRV51004']

'Nonsp abn find-gi tract, Muscle weakness-general, Hallucinations, Acute kidney failure NOS, Symbolic dysfunction NEC, , , , , ; Mitral valve disorder, Cardiomegaly, Disorder of thyroid NOS, , , , , , , ; Fam hx genet dis carrier, Long-term use meds NEC, Hypertension NOS, Follow-up surgery NEC, , , , , , ; Hypertension NOS, Acute bronchitis, , , , , , , , ; Abdmnal pain periumbilic, Bariatric surgery status, Flatul/eructat/gas pain, Thrush, , , , , , ; Hyperlipidemia NEC/NOS, DMII wo cmp nt st uncntr, Abn blood chemistry NEC, , , , , , , ; Attn rem nonsurg dressng, , , , , , , , , ; Mal neo retroperitoneum, Cardiac dysrhythmias NEC, Pulm embol/infarct NEC, Other specfd counseling, , , , , , ; Mal neo breast low-outer, Screen malig neop-colon, , , , , , , , ; DMII hprosmlr uncontrold, Malign neopl breast NEC, Fit/adjust insulin pump, Hypothyroidism NOS, Benign hypertension, Hypertension NOS, , , , ; Screen-blood dis NOS, V726, , , , , , , , ; Dialysis encounter, NEC, Sec hyperparathyrd-

####Generate the embeddings for these features using BERT transformer

In [19]:
# Run if sentence-transformers not installed
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 10.2 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=ea72643241f8255ef845dcff5f7ef2088aebafead44f07bad6a583c9a5e4395b
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [20]:
from sentence_transformers import SentenceTransformer
from transformers import BertTokenizer, BertModel
import torch
import time
import pickle

In [21]:
model = SentenceTransformer('distilbert-base-nli-mean-tokens')

.gitattributes:   0%|          | 0.00/690 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.99k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/550 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/450 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

In [22]:
# Function to convert the embeddings to a dataframe
def make_df(embeddings, source):
  idx = source.index

  final = pd.DataFrame(embeddings, index=idx)
  return final

####Generate embeddings for the training set

In [23]:
# Function to perform encoding
start = time.time()
embed_train = model.encode(grouped_diag_train, show_progress_bar=True)
embed_df_train = make_df(embed_train, grouped_diag_train)
embed_df_train.to_pickle("/content/embed_df_train.pkl")
end = time.time()
print(f"Diagnosis codes for training set embedded: Time taken is {end-start} seconds")

Batches:   0%|          | 0/136 [00:00<?, ?it/s]

Diagnosis codes for training set embedded: Time taken is 997.5700297355652 seconds


In [24]:
embed_df_train.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,...,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610,611,612,613,614,615,616,617,618,619,620,621,622,623,624,625,626,627,628,629,630,631,632,633,634,635,636,637,638,639,640,641,642,643,644,645,646,647,648,649,650,651,652,653,654,655,656,657,658,659,660,661,662,663,664,665,666,667,668,669,670,671,672,673,674,675,676,677,678,679,680,681,682,683,684,685,686,687,688,689,690,691,692,693,694,695,696,697,698,699,700,701,702,703,704,705,706,707,708,709,710,711,712,713,714,715,716,717,718,719,720,721,722,723,724,725,726,727,728,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767
Provider,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
PRV51004,-0.851374,-0.296298,1.004773,-0.569999,-0.838826,-0.309694,0.352073,0.190344,0.055761,-0.341353,0.359041,0.600373,-0.686077,0.882949,0.777989,-0.235487,0.413462,-0.486286,-0.159975,0.246992,0.054514,-0.035308,0.415088,1.966550,-1.060282,0.136390,1.196987,0.267912,0.034872,-0.426109,0.844477,-0.176323,-0.811470,0.320338,0.294035,-0.404453,0.889065,0.262198,0.390689,0.208982,0.194198,-0.790692,0.410620,0.535363,-0.469076,-0.023397,-1.013141,-0.204697,-1.089434,-0.578835,-0.184088,0.241313,0.673976,-0.537729,0.021185,-0.092875,0.153301,-0.524885,0.016039,0.408290,0.922320,-0.160902,-0.213943,-0.894689,-0.038371,-0.499081,0.004085,0.405514,0.162633,-0.033707,0.488515,0.529485,0.008521,-0.664482,0.101011,0.484321,-0.145332,-1.071562,0.347730,-0.399491,-0.177154,-0.553071,0.075514,0.420319,-0.443803,-0.057090,0.423329,-0.425427,0.452261,0.946955,-0.094647,-0.393627,0.516986,-0.362394,0.079792,-0.823858,0.715754,-0.749590,-0.273154,0.093035,-0.266226,0.390541,0.065422,0.000949,-0.039534,-0.582464,0.559755,0.511469,-0.371689,0.618258,-0.106612,0.480173,-0.425154,-0.855151,-0.187052,-1.728539,0.348804,0.481984,-0.426413,0.433436,0.423835,1.101122,1.103548,-0.098740,0.344878,0.967906,0.573985,0.510749,-0.296572,-0.514804,-0.071048,0.499897,-0.400414,-0.537147,0.842560,-0.380925,-0.133118,-0.119719,-0.569359,0.411558,0.037620,-0.185977,0.421850,0.071652,-0.091542,-0.423163,-1.508608,0.012443,0.756364,0.044247,-1.264607,0.286100,-0.051517,-0.313356,0.148310,0.180119,0.463938,-0.387465,1.135949,-0.035682,-0.112695,0.278624,0.533824,0.164759,0.109457,-0.958087,-0.372574,-0.

In [26]:
embed_df_train.shape

(4328, 768)

####Generate embeddings for the testing set

In [27]:
# Function to perform encoding on validation data
start = time.time()
embed_val = model.encode(grouped_diag_val, show_progress_bar=True)
embed_df_val = make_df(embed_val, grouped_diag_val)
embed_df_val.to_pickle("/content/embed_df_val.pkl")
end = time.time()
print(f"Diagnosis codes for validation set embedded: Time taken is {end-start} seconds")

Batches:   0%|          | 0/34 [00:00<?, ?it/s]

Diagnosis codes for validation set embedded: Time taken is 243.44031381607056 seconds


In [28]:
embed_df_val.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,...,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610,611,612,613,614,615,616,617,618,619,620,621,622,623,624,625,626,627,628,629,630,631,632,633,634,635,636,637,638,639,640,641,642,643,644,645,646,647,648,649,650,651,652,653,654,655,656,657,658,659,660,661,662,663,664,665,666,667,668,669,670,671,672,673,674,675,676,677,678,679,680,681,682,683,684,685,686,687,688,689,690,691,692,693,694,695,696,697,698,699,700,701,702,703,704,705,706,707,708,709,710,711,712,713,714,715,716,717,718,719,720,721,722,723,724,725,726,727,728,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767
Provider,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
PRV51001,-0.402748,-0.332463,0.627804,-0.802203,-0.444155,-0.186443,0.288258,-0.104364,-0.259987,0.069541,0.573037,0.381283,-1.053048,0.631668,0.925904,0.047906,0.158230,-0.049958,0.083542,0.246459,0.212979,0.238966,0.645884,1.873321,-0.892961,-0.215857,1.041883,-0.137000,-0.047338,-0.457215,0.852731,-0.110503,-0.656074,-0.263684,0.236998,-0.822480,0.638983,0.047414,0.259939,0.304864,-0.233299,-1.127319,-0.057245,0.631765,0.135149,0.059537,-0.496046,0.066745,-0.800758,-0.385695,-0.172966,-0.304296,0.561001,-0.220621,0.036721,0.013197,0.435632,-0.782005,-0.129330,0.424158,0.604977,-0.588505,-0.538611,-0.861615,0.427758,-0.482581,-0.114035,0.873245,-0.056035,0.388140,0.682230,0.270279,0.146039,-0.551065,0.098328,0.429003,0.355718,-1.218191,-0.026580,-0.532071,-0.311561,-0.647422,-0.310202,0.635001,-0.196782,-0.282354,0.467675,-0.314677,0.599303,0.902809,-0.234946,-0.264104,0.362585,-0.443202,0.419860,-0.663004,0.601518,-0.524833,-0.421268,0.182054,-0.214444,0.910248,0.250447,0.321037,-0.093830,-0.684288,1.027311,0.570915,-0.070167,0.182860,0.253499,0.414891,-0.300295,-0.567565,-0.180211,-0.924387,0.350358,0.473993,-0.020046,0.409607,0.039956,1.070002,1.048381,-0.499776,0.262652,0.923554,0.034637,0.560270,0.209735,-0.834328,0.212526,0.506157,-0.327770,-0.518666,0.138614,-0.281040,-0.056437,0.361106,-0.208191,0.075117,-0.451668,-0.139138,0.730073,-0.047914,-0.391704,-0.200905,-1.950929,0.263489,0.264746,-0.261431,-1.011480,0.171987,0.068773,-0.596799,-0.136524,0.498408,0.178453,-0.489504,1.059344,-0.307733,0.229013,-0.416501,0.566768,0.282230,0.502430,-0.850712,-0.408247,-